# Getting data from APIs

Você não pode construir um modelo sem dados, certo? Em projetos anteriores, trabalhamos com dados armazenados em arquivos (como um CSV) ou bancos de dados (como SQL). Neste projeto, vamos obter nossos dados de um servidor web usando uma API.

Então, nesta lição, vamos aprender o que é uma API e como extrair dados de uma. Também vamos trabalhar na transformação de nossos dados em um formato gerenciável. Vamos lá!

In [1]:
import pandas as pd
import requests

# Acessando APIs através de uma URL

Nesta lição, vamos extrair informações do mercado de ações da API [AlphaVantage](https://alphavantage.co/). Para entender como uma API funciona, considere a URL abaixo.

Tire um momento para ler o texto do link em si, depois clique nele e examine os dados que aparecem no seu navegador. Qual é o formato dos dados? Quais dados estão incluídos? Como eles estão organizados?

https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=demo

Observe que esta URL possui vários componentes. Vamos analisá-los um por um.

| URL | Componente |
|:--- | :-------- |
| `https://www.alphavantage.co` | Este é o **hostname** ou **URL base**. É o endereço web do servidor onde podemos obter nossos dados de ações. |
| `/query` | Este é o **caminho**. A maioria das APIs tem várias operações diferentes que podem ser feitas. O caminho é o nome da operação específica que queremos acessar. |
| `?` | Este ponto de interrogação indica que tudo o que segue na URL é um **parâmetro**. Cada parâmetro é separado por um caractere `&`. Esses parâmetros fornecem informações adicionais que irão modificar o comportamento da operação. Isso é semelhante à forma como passamos **argumentos** para funções em Python. |
| `function=TIME_SERIES_DAILY` | Nosso primeiro parâmetro usa a palavra-chave `function`. O valor é `TIME_SERIES_DAILY`. Neste caso, estamos pedindo dados de ações **diários**. |
| `symbol=IBM` | Nosso segundo parâmetro usa a palavra-chave `symbol`. Portanto, estamos pedindo dados de uma ação cujo [**ticker symbol**](https://en.wikipedia.org/wiki/Ticker_symbol) é `IBM`. |
| `apikey=demo` | Da mesma forma que você precisa de uma senha para acessar alguns sites, uma **chave de API** ou **token de API** é a senha que você usará para acessar a API. |

Agora que temos uma noção dos componentes da URL que obtém informações do AlphaVantage, vamos criar a nossa própria para uma ação diferente.

### Exercício:
Usando a URL acima como modelo, crie uma nova URL para obter os dados da [Ambuja Cement](https://www.ambujacement.com/). com o ticker sendo `"AMBUJACEM.BSE"`.


In [2]:
url = (
    "https://www.alphavantage.co"    # URL base
    "/query?"                        # caminho
    "function=TIME_SERIES_DAILY&"    # param
    "symbol=AMBUJACEM.BSE&"          # param
    "apikey=demo"                    # param
)

print("url type:", type(url))
url

url type: <class 'str'>


'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AMBUJACEM.BSE&apikey=demo'

Oh não! Um problema. Parece que precisamos da nossa própria chave de API para acessar os dados.

Como você pode imaginar, uma chave de API é uma informação que deve ser mantida em sigilo, então é uma má ideia incluí-la no código da nossa aplicação. Quando se trata de informações sensíveis como esta, desenvolvedores e cientistas de dados as armazenam como uma [variável de ambiente](https://en.wikipedia.org/wiki/Environment_variable) que é mantida em um arquivo `.env`.

### Exercício:
Obtenha sua chave de API e a salve em seu arquivo `.env`.

Agora que armazenamos nossa chave de API, precisamos importá-la para o nosso código. Isso é comumente feito criando um módulo `config`.

### Exercício:
Importe a variável `settings` do módulo `config`. Em seguida, use o comando `dir` para ver quais atributos ela possui.

In [10]:
# Import settings
# !pip install pydantic
from config import settings

# Use `dir` to list attributes
dir(settings)

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_validator__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',

In [ ]:
settings.alpha_api_key

### Exercício:
Crie uma nova URL para `"AMBUJACEM.BSE"`:


In [11]:
url = (
    "https://www.alphavantage.co"
    "/query?"
    "function=TIME_SERIES_DAILY&"
    "symbol=AMBUJACEM.BSE&"
    f"apikey={settings.alpha_api_key}"
)

print("url type:", type(url))
url

url type: <class 'str'>


'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AMBUJACEM.BSE&apikey=JWK8Z958ODBXS114'

Está funcionando! Acontece que há muitos mais parâmetros. Vamos construir nossa URL para incluí-los.

### Exercício:
Vá até a documentação da [API AlphaVantage Time Series Daily](https://www.alphavantage.co/documentation/#daily). Amplie sua URL para incorporar todos os parâmetros listados na documentação. Além disso, para tornar sua URL mais dinâmica, crie nomes de variáveis para todos os parâmetros que podem ser adicionados à URL.

In [17]:
ticker = "AMBUJACEM.BSE"
output_size = "compact"
data_type = "json"

url = (
    "https://www.alphavantage.co"
    "/query?"
    "function=TIME_SERIES_DAILY&"
    f"symbol={ticker}&"
    f"outputsize={output_size}&"
    f"datatype={data_type}&"
    f"apikey={settings.alpha_api_key}"
)

print("url type:", type(url))
url

url type: <class 'str'>


'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AMBUJACEM.BSE&outputsize=compact&datatype=json&apikey=JWK8Z958ODBXS114'

# Acessando APIs Através de uma Solicitação

Vimos como acessar a API AlphaVantage clicando em uma URL, mas isso não funcionará para a aplicação que estamos construindo neste projeto, pois apenas humanos clicam em URLs. Programas de computador acessam APIs fazendo **solicitações**. Vamos construir nossa primeira solicitação usando a URL que criamos na tarefa anterior.

### Exercício:
Use a biblioteca `requests` para fazer uma solicitação `get` à URL que você criou na tarefa anterior. Atribua a resposta à variável `response`.

In [18]:
response = requests.get(url=url)

print("response type:", type(response))

response type: <class 'requests.models.Response'>


Isso nos informa que tipo de resposta recebemos, mas não nos diz nada sobre o que isso significa. Se quisermos descobrir quais tipos de dados estão realmente *na* resposta, precisaremos usar o comando `dir`.

### Exercício:
Use o comando `dir` para ver quais atributos e métodos a variável `response` possui.

In [20]:
# Use `dir` on your `response`
dir(response)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

O comando `dir` retorna uma lista e, como você pode ver, há muitas possibilidades aqui! Por enquanto, vamos nos concentrar em dois atributos: `status_code` e `text`.

Começaremos com `status_code`. Sempre que você faz uma chamada para uma URL, a resposta inclui um [código de status HTTP](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes) que pode ser acessado com o atributo `status_code`. Vamos ver qual é o nosso.

### Exercício:
Atribua o código de status da sua `response` à variável `response_code`.

In [34]:
response_code = response.status_code

print("code type:", type(response_code))
print(response_code)

code type: <class 'int'>
200


Traduzido para o inglês, `200` significa "OK". É a resposta padrão para uma solicitação HTTP bem-sucedida. Em outras palavras, funcionou! Recebemos com sucesso os dados da API AlphaVantage.

Agora vamos dar uma olhada no `text`.

### Exercício:
Atribua o texto da sua `response` à variável `response_text`.

In [35]:
response_text = response.text

print("response_text type:", type(response_text))
print(response_text[:200])

response_text type: <class 'str'>
{
    "Meta Data": {
        "1. Information": "Daily Prices (open, high, low, close) and Volumes",
        "2. Symbol": "AMBUJACEM.BSE",
        "3. Last Refreshed": "2024-10-15",
        "4. Output 


Essa string parece com os dados que vimos anteriormente em nosso navegador quando clicamos na URL no exercício 5. Mas não podemos trabalhar com dados estruturados como JSON quando estão como uma string. Em vez disso, precisamos que eles estejam em um dicionário.

### Exercício:
Use o método `json` para acessar uma versão em dicionário dos dados. Atribua-a à variável chamada `response_data`.

In [36]:
response_data = response.json()

print("response_data type:", type(response_data))

response_data type: <class 'dict'>


Vamos verificar se os dados estão estruturados da mesma forma que vimos em nosso navegador.

### Exercício:
Imprima as chaves de `response_data`. Elas são o que você esperava?

In [37]:
# Print `response_data` keys
response_data.keys()

dict_keys(['Meta Data', 'Time Series (Daily)'])

Agora vamos olhar os dados que estão atribuídos à chave `"Time Series (Daily)"`.

### Exercício:
Atribua o valor da chave `"Time Series (Daily)"` à variável `stock_data`. Em seguida, examine os dados de um dos dias em `stock_data`.

In [ ]:
# Extract `"Time Series (Daily)"` value from `response_data`
stock_data = ...

print("stock_data type:", type(stock_data))

# Extract data for one of the days in `stock_data`



Agora que sabemos como os dados estão organizados quando os extraímos da API, vamos transformá-los em um DataFrame para torná-los mais gerenciáveis.

### Exercício:
Leia os dados de `stock_data` em um DataFrame chamado `df_ambuja`. Certifique-se de que todos os seus tipos de dados estão corretos!

In [ ]:
df_ambuja = ...

print("df_ambuja shape:", df_ambuja.shape)
print()
print(df_ambuja.info())
df_ambuja.head(10)

Você notou que o índice para `df_ambuja` não tem uma entrada para todos os dias? Considerando que se trata de dados do mercado de ações, por que você acha que isso acontece?

No geral, isso parece muito bom, mas há alguns problemas: o tipo de dados das datas e o formato dos cabeçalhos. Vamos corrigir as datas primeiro. No momento, as datas são strings; para que o restante do nosso código funcione, precisaremos criar um `DatetimeIndex` adequado.

### Exercício:
Transforme o índice de `df_ambuja` em um `DatetimeIndex` com o nome `"date"`.

In [ ]:
# Convert `df_ambuja` index to `DatetimeIndex`


# Name index "date"


print(df_ambuja.info())
df_ambuja.head()

__Nota:__ as linhas em `df_ambuja` estão ordenadas de forma <b>decrescente</b>, com a data mais recente no topo. Isso será vantajoso quando armazenarmos e recuperarmos os dados do nosso banco de dados da aplicação, mas precisaremos ordená-los de forma <b>crescente</b> antes de podermos usá-los para treinar um modelo.

Ok! Agora que as datas estão corrigidas, vamos lidar com os cabeçalhos. Não há nada realmente *errado* com eles, mas aqueles números os fazem parecer um pouco inacabados. Vamos nos livrar deles.

### Exercício:
Remova a numeração dos nomes das colunas de `df_ambuja`.

In [ ]:
# Remove numbering from `df_ambuja` column names
df_ambuja.columns = ...

print(df_ambuja.info())
df_ambuja.head()

# Programação Defensiva

Programação defensiva é a prática de escrever código que continuará a funcionar, mesmo que algo dê errado. Nunca conseguiremos prever todos os problemas que as pessoas podem encontrar com nosso código, mas podemos tomar medidas para garantir que as coisas não desmoronem sempre que um desses problemas acontecer.

Até agora, fizemos solicitações de API onde tudo funciona. Mas erros de codificação e problemas com servidores são comuns, e podem causar grandes problemas em um projeto de ciência de dados. Vamos ver como nossa `response` muda quando introduzimos erros comuns em nosso código.

### Exercício:
Volte ao exercício 5 e mude a primeira parte da sua URL. Em vez de `"query"`, use `"search"` (um caminho que não existe). Em seguida, execute novamente seu código para todas as tarefas que se seguem. O que muda? O que permanece o mesmo?

Sabemos o que acontece quando tentamos acessar um endereço inválido. Mas e quando acessamos o *caminho correto* com um símbolo de ação *inválido*?

### Exercício:
Volte ao exercício 5 e mude o símbolo da ação de `"AMBUJACEM.BSE"` para `"RAMBUJACEM.BSE"` (uma empresa que não existe). Em seguida, execute novamente seu código para todas as tarefas que se seguem. Novamente, observe o que muda e o que permanece o mesmo.

Vamos formalizar nosso processo de extração e transformação para a API AlphaVantage em uma função reproduzível.

### Exercício:
Construa uma função `get_daily` que obtenha dados da API AlphaVantage e retorne um DataFrame limpo. Use a docstring como guia. Quando estiver satisfeito com o resultado, envie seu trabalho para o avaliador.

In [ ]:
def get_daily():

    """Get daily time series of an equity from AlphaVantage API.

    Parameters
    ----------
    ticker : str
        The ticker symbol of the equity.
    output_size : str, optional
        Number of observations to retrieve. "compact" returns the
        latest 100 observations. "full" returns all observations for
        equity. By default "full".

    Returns
    -------
    pd.DataFrame
        Columns are 'open', 'high', 'low', 'close', and 'volume'.
        All are numeric.
    """
    # Create URL (8.1.5)


    # Send request to API (8.1.6)


    # Extract JSON data from response (8.1.10)


    # Read data into DataFrame (8.1.12 & 8.1.13)


    # Convert index to `DatetimeIndex` named "date" (8.1.14)


    # Remove numbering from columns (8.1.15)


    # Return DataFrame
    return df

In [ ]:
# Test your function
df_ambuja = get_daily(ticker="AMBUJACEM.BSE")

print(df_ambuja.info())
df_ambuja.head()

Como essa função lida com os dois erros que exploramos nesta seção? Nosso primeiro erro, uma URL inválida, é algo com o qual não precisamos nos preocupar. Não importa o que o usuário insira nesta função, a URL sempre estará correta. Mas veja o que acontece quando o usuário insere um símbolo de ação inválido. Qual é a mensagem de erro? Isso ajudaria o usuário a localizar seu erro?

### Exercício:
Adicione uma cláusula `if` à sua função `get_daily` para que ela lance uma `Exception` quando um usuário fornecer um símbolo de ação inválido. Certifique-se de que a mensagem de erro seja informativa.

In [ ]:
# Test your Exception
df_test = get_daily(ticker="ABUJACEM.BSE")

Certo! Agora temos todas as ferramentas necessárias para obter os dados para o nosso projeto. Na próxima lição, tornaremos nosso código AlphaVantage mais reutilizável, criando um módulo `data` com definições de classe. Também criaremos o código necessário para armazenar e ler esses dados do nosso banco de dados de aplicação.

# Desenvolvimento Orientado a Testes (TDD)

Na lição anterior, aprendemos como obter dados de uma API. Nesta lição, temos dois objetivos. Primeiro, vamos pegar o código que usamos para acessar a API e construir uma classe `AlphaVantageAPI`. Isso nos permitirá reutilizar nosso código. Segundo, criaremos uma classe `SQLRepository` que nos ajudará a carregar nossos dados de ações em um banco de dados SQLite e depois extraí-los para uso posterior. Além disso, construiremos esse código usando uma técnica chamada **desenvolvimento orientado a testes**, onde usaremos instruções `assert` para garantir que tudo esteja funcionando corretamente. Assim, evitaremos problemas mais tarde ao construir nossa aplicação.

In [ ]:
%load_ext autoreload
%load_ext sql
%autoreload 2

import sqlite3
import matplotlib.pyplot as plt
import pandas as pd
from config import settings

# Construindo Nosso Módulo de Dados

Para nossa aplicação, vamos manter todas as classes que usamos para extrair, transformar e carregar dados em um único módulo que chamaremos de `data`.

## Classe AlphaVantage API

Vamos começar pegando o código que criamos na última lição e incorporá-lo em uma classe que será responsável por obter dados da API AlphaVantage.

### Exercício:
No módulo `data`, crie uma definição de classe para `AlphaVantageAPI`. Por enquanto, certifique-se de que ela tenha um método `__init__` que anexe sua chave de API como o atributo `__api_key`. Depois de terminar, importe a classe abaixo e crie uma instância dela chamada `av`.

In [ ]:
# Import `AlphaVantageAPI`


# Create instance of `AlphaVantageAPI` class
av = ...

print("av type:", type(av))

Lembre-se da função `get_daily` que fizemos na última lição? Agora vamos transformá-la em um método de classe.

### Exercício:
Crie um método `get_daily` para sua classe `AlphaVantageAPI`. Assim que terminar, use a célula abaixo para buscar os dados das ações da empresa de energia renovável [Suzlon](https://www.suzlon.com/) e atribua-os ao DataFrame `df_suzlon`.

In [ ]:
# Define Suzlon ticker symbol
ticker = "SUZLON.BSE"

# Use your `av` object to get daily data
df_suzlon = ...

print("df_suzlon type:", type(df_suzlon))
print("df_suzlon shape:", df_suzlon.shape)
df_suzlon.head()

Certo! A próxima coisa que precisamos fazer é testar nosso novo método para garantir que ele funcione da maneira que desejamos. Normalmente, esses tipos de testes são escritos *antes* de escrever o método, mas, neste primeiro caso, faremos o contrário para entender melhor como as instruções de assert funcionam.

### Exercício:
Crie quatro instruções `assert` para testar a saída do seu método `get_daily`. Use os comentários abaixo como guia.

In [ ]:
# Does `get_daily` return a DataFrame?


# Does DataFrame have 5 columns?


# Does DataFrame have a DatetimeIndex?


# Is the index name "date"?


### Exercício:
Crie mais dois testes para a saída do seu método `get_daily`. Use os comentários abaixo como guia.

In [ ]:
# Does DataFrame have correct column names?


# Are columns correct data type?


Ok! Agora que nossa classe `AlphaVantageAPI` está pronta para obter dados, vamos focar na classe que precisaremos para armazenar nossos dados em nosso banco de dados SQLite.

## Classe Repositório SQL

Não seria eficiente se nosso aplicativo precisasse obter dados da API AlphaVantage toda vez que quiséssemos explorar nossos dados ou construir um modelo, então precisaremos armazenar nossos dados em um banco de dados. Como nossos dados são altamente estruturados (cada DataFrame que extraímos da AlphaVantage sempre terá as mesmas cinco colunas), faz sentido usar um banco de dados SQL.

Usaremos o SQLite para nosso banco de dados. Para consistência, esse banco de dados sempre terá o mesmo nome, que armazenamos em nosso arquivo `.env`.

### Exercício:
Conecte-se ao banco de dados cujo nome está armazenado no arquivo `.env` deste projeto. Certifique-se de definir o argumento `check_same_thread` como `False`. Atribua a conexão à variável `connection`.

In [ ]:
connection = ...

print("connection type:", type(connection))

Temos uma conexão, e agora precisamos começar a construir a classe que irá gerenciar todas as nossas transações com o banco de dados. Com esta classe, no entanto, vamos criar nossos testes *antes* de escrever a definição da classe.

### Exercício:
Escreva dois testes para a classe `SQLRepository`, usando os comentários abaixo como guia.

In [ ]:
# Import class definition


# Create instance of class
repo = ...

# Does `repo` have a "connection" attribute?


# Is the "connection" attribute a SQLite `Connection`?


__Dica:__ Você não poderá executar este bloco de código ☝️ até completar a tarefa abaixo. 👇

### Exercício:
Crie uma definição para sua classe `SQLRepository`. Por enquanto, complete apenas o método `__init__`. Assim que terminar, use o código que você escreveu na tarefa anterior para testá-la.

O próximo método que precisamos para a classe `SQLRepository` é um que nos permita armazenar informações. No jargão SQL, isso é geralmente referido como **inserir** tabelas no banco de dados.

### Exercício:
Adicione um método `insert_table` à sua classe `SQLRepository`. Como guia, use as instruções `assert` abaixo e o docstring no módulo `data`. Quando terminar, execute a célula abaixo para verificar seu trabalho.

In [ ]:
response = repo.insert_table(table_name=ticker, records=df_suzlon, if_exists="replace")

# Does your method return a dictionary?
assert isinstance(response, dict)

# Are the keys of that dictionary correct?
assert sorted(list(response.keys())) == ["records_inserted", "transaction_successful"]

Se nosso método estiver passando as instruções `assert`, sabemos que ele está retornando um registro da transação no banco de dados, mas ainda precisamos verificar se os dados foram realmente adicionados ao banco de dados.

### Exercício:
Escreva uma consulta SQL para obter as **cinco primeiras linhas** da tabela de dados da Suzlon que você acabou de inserir no banco de dados.

In [ ]:

%sql sqlite:///

In [ ]:
%%sql



Podemos inserir dados no nosso banco de dados, mas não devemos esquecer que também precisamos ler dados dele. A leitura será um pouco mais complexa do que a inserção, então vamos começar escrevendo o código neste notebook antes de incorporá-lo à nossa classe `SQLRepository`.

### Exercício:
Primeiro, escreva uma consulta SQL para obter **todos** os dados da Suzlon. Em seguida, use o pandas para extrair os dados do banco de dados e ler em um DataFrame, chamado `df_suzlon_test`.

In [ ]:
sql = ...
df_suzlon_test = ...

print("df_suzlon_test type:", type(df_suzlon_test))
print()
print(df_suzlon_test.info())
df_suzlon_test.head()

Agora que sabemos como ler uma tabela do nosso banco de dados, vamos transformar nosso código em uma função adequada. Mas, como estamos fazendo design reverso, precisamos começar com nossos testes.

### Exercício:
Complete as declarações `assert` abaixo para testar sua função `read_table`. Use os comentários como guia.

In [ ]:
# Assign `read_table` output to `df_suzlon`
df_suzlon = read_table(table_name="SUZLON.BSE", limit=2500)  # noQA F821

# Is `df_suzlon` a DataFrame?


# Does it have a `DatetimeIndex`?


# Is the index named "date"?


# Does it have 2,500 rows and 5 columns?


# Are the column names correct?


# Are the column data types correct?


# Print `df_suzlon` info
print("df_suzlon shape:", df_suzlon.shape)
print()
print(df_suzlon.info())
df_suzlon.head()

__Dica:__ Você não poderá executar este bloco de código ☝️ até completar a tarefa abaixo. 👇

### Exercício:
Expanda o código que você escreveu acima para completar a função `read_table` abaixo. Use a docstring como um guia.

__Dica:__ Lembre-se de que armazenamos nossos dados ordenados de forma <b>decrescente</b> pela data. Isso definitivamente facilitará a implementação do <code>read_table</code>!

In [ ]:
def read_table():

    """Read table from database.

    Parameters
    ----------
    table_name : str
        Name of table in SQLite database.
    limit : int, None, optional
        Number of most recent records to retrieve. If `None`, all
        records are retrieved. By default, `None`.

    Returns
    -------
    pd.DataFrame
        Index is DatetimeIndex "date". Columns are 'open', 'high',
        'low', 'close', and 'volume'. All columns are numeric.
    """
    # Create SQL query (with optional limit)
    sql = ...


    # Retrieve data, read into DataFrame
    df = ...

    # Return DataFrame
    return df

### Exercício:
Transforme a função `read_table` em um método da sua classe `SQLRepository`.

### Exercício:
Volte à tarefa 11 e altere o código para que você teste o método da sua classe em vez da função do notebook.

Excelente! Temos tudo o que precisamos para obter dados do AlphaVantage, salvar esses dados em nosso banco de dados e acessá-los mais tarde. Agora é hora de fazer uma pequena análise exploratória para comparar as ações das duas empresas para as quais temos dados.

# Comparar Retorno de Ações

Já temos os dados da Suzlon Energy em nosso banco de dados, mas precisamos adicionar os dados da Ambuja Cement antes de podermos comparar as duas ações.

### Exercício:
Use as instâncias das classes `AlphaVantageAPI` e `SQLRepository` que você criou nesta lição (`av` e `repo`, respectivamente) para obter os dados de ações da Ambuja Cement e armazená-los no banco de dados.

In [ ]:
ticker = "AMBUJACEM.BSE"

# Get Ambuja data using `av`
ambuja_records = ...

# Insert `ambuja_records` database using `repo`
response = ...

response

Vamos dar uma olhada nos dados para garantir que estamos obtendo o que precisamos.

### Exercício:
Usando o método `read_table` que você adicionou ao seu `SQLRepository`, extraia as 2.500 linhas mais recentes de dados da Ambuja Cement do banco de dados e atribua o resultado a `df_ambuja`.

In [ ]:
ticker = "AMBUJACEM.BSE"
df_ambuja = ...

print("df_ambuja type:", type(df_ambuja))
print("df_ambuja shape:", df_ambuja.shape)
df_ambuja.head()

Já passamos bastante tempo observando esses dados, mas o que eles realmente representam? Acontece que o mercado de ações é muito parecido com qualquer outro mercado: as pessoas compram e vendem bens. Os preços desses bens podem subir ou descer dependendo de fatores como oferta e demanda. No caso de um mercado de ações, os bens sendo vendidos são ações (também chamadas de "equities" ou "securities"), que representam uma participação na propriedade de uma corporação.

Durante cada dia de negociação, o preço de uma ação muda, então, quando estamos avaliando se uma ação pode ser um bom investimento, observamos quatro tipos de números: abertura (open), máximo (high), mínimo (low), fechamento (close) e volume (volume). **Abertura (Open)** é exatamente o que parece: o preço de venda de uma ação quando o mercado abre para o dia. Da mesma forma, **fechamento (Close)** é o preço de venda de uma ação quando o mercado fecha no final do dia, e **máximo (High)** e **mínimo (Low)** são os preços máximo e mínimo de uma ação ao longo do dia. **Volume** é o número de ações de uma determinada empresa que foram compradas e vendidas naquele dia. De modo geral, uma empresa cujas ações tiveram um grande volume de negociação verá mais variação de preço ao longo do dia do que uma empresa cujas ações foram negociadas de forma mais leve.

Vamos visualizar como o preço da Ambuja Cement mudou na última década.

### Exercício:
Plote o preço de fechamento de `df_ambuja`. Certifique-se de rotular seus eixos e incluir uma legenda.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
# Plot `df_ambuja` closing price


# Label axes



# Add legend


Vamos adicionar o preço de fechamento da Suzlon ao nosso gráfico para que possamos comparar os dois.

### Exercício:
Crie um gráfico que mostre os preços de fechamento de `df_suzlon` e `df_ambuja`. Novamente, rotule seus eixos e inclua uma legenda.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
# Plot `df_suzlon` and `df_ambuja`



# Label axes



# Add legend


Olhando para este gráfico, podemos concluir que a Ambuja Cement é uma ação "melhor" do que a Suzlon Energy, pois seu preço é mais alto. Mas o preço é apenas um fator que um investidor deve considerar ao criar uma estratégia de investimento. O que é definitivamente verdade é que é difícil fazer uma comparação direta entre essas duas ações, pois há uma diferença de preço tão grande.

Uma maneira de os investidores compararem ações é observando seus **retornos**. Um retorno é a mudança de valor em um investimento, representada como uma porcentagem. Então, vamos analisar os retornos diários de nossas duas ações.

### Exercício:
Adicione uma coluna `"return"` ao `df_ambuja` que mostre a mudança percentual na coluna `"close"` de um dia para o outro.

__Dica:__ Nossos dois DataFrames estão ordenados de forma <b>decrescente</b> por data, mas você precisará garantir que estejam ordenados de forma <b>crescente</b> para calcular seus retornos.

In [ ]:
# Sort DataFrame ascending by date


# Create "return" column


print("df_ambuja shape:", df_ambuja.shape)
print(df_ambuja.info())
df_ambuja.head()

### Exercício:
Adicione uma coluna `"return"` ao `df_suzlon` que mostre a variação percentual na coluna `"close"` de um dia para o outro.

In [ ]:
# Sort DataFrame ascending by date


# Create "return" column


print("df_suzlon shape:", df_suzlon.shape)
print(df_suzlon.info())
df_suzlon.head()

Agora vamos plotar os retornos das nossas duas empresas e ver como as duas se comparam.

### Exercício:
Plote os retornos de `df_suzlon` e `df_ambuja`. Certifique-se de rotular seus eixos e usar uma legenda.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
# Plot returns for `df_suzlon` and `df_ambuja`



# Label axes



# Add legend


Sucesso! Ao representar os retornos como uma porcentagem, conseguimos comparar duas ações que têm preços muito diferentes. Mas o que essa visualização está nos dizendo? Podemos ver que os retornos da Suzlon têm uma variação maior. Observamos grandes ganhos e grandes perdas. Em contraste, a variação da Ambuja é mais estreita, o que significa que o preço não flutua tanto.

Outro nome para essa flutuação diária nos retornos é chamado de [**volatilidade**](https://en.wikipedia.org/wiki/Volatility_(finance)), que é outro fator importante para os investidores. Portanto, na próxima lição, aprenderemos mais sobre volatilidade e, em seguida, construiremos um modelo de séries temporais para prever isso.